Creating OpenAI Gym Environment   
[Part 1](https://gerardmaggiolino.medium.com/creating-openai-gym-environments-with-pybullet-part-1-13895a622b24)  |  [Part 2](https://gerardmaggiolino.medium.com/creating-openai-gym-environments-with-pybullet-part-2-a1441b9a4d8e)  
[Video 1](https://urldefense.com/v3/__https://www.youtube.com/watch?v=tozzb6xC1LI__;!!Mih3wA!Buejbwp2bgHIMcIvfX7xWLQr6D-qXoKnlnkY_rIRrQEL0KTlHL8E_qtZHCSQTc3S3kCAQtqNhRyLrw$)  |  [Video 2](https://www.youtube.com/watch?v=DZ5Px-ocelw)

In [1]:
import pybullet as p
import pybullet_data

pybullet build time: Apr  8 2023 22:18:57


In [2]:
client = p.connect(p.GUI) # Can alternatively pass in p.DIRECT 
p.setAdditionalSearchPath(pybullet_data.getDataPath()) # Gives paths to data like existing URDFs
p.setGravity(0, 0, -9.8, physicsClientId=client)

2023-05-07 19:28:56.832 Python[68890:2214085] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/j3/7jchtq_n7cz4zm5b85nj95lc0000gn/T/org.python.python.savedState


Version = 4.1 ATI-4.9.51
Vendor = ATI Technologies Inc.
Renderer = AMD Radeon Pro 5500M OpenGL Engine
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


In [3]:
# These add controllable parameters to the GUI
# pause = p.addUserDebugParameter('Pause', 1, 0, 0)
angle = p.addUserDebugParameter('Steering', -0.5, 0.5, 0)
throttle = p.addUserDebugParameter('Throttle', 0, 20, 0)

In [4]:
original_simulation = False
if original_simulation:
    carId = p.loadURDF("simplecar.urdf")
    while p.isConnected():
        pos, ori = p.getBasePositionAndOrientation(carId)
        p.applyExternalForce(carId, 0, [1, 0, 0], pos, p.WORLD_FRAME) # p.WORLD_FRAME or p.LINK_FRAME
        p.stepSimulation()


In [5]:
plane = p.loadURDF('simpleplane.urdf')
carId = p.loadURDF("simplecar.urdf", [0, 0, 0.1])
number_of_joints = p.getNumJoints(carId)
for joint_number in range(number_of_joints):
    info = p.getJointInfo(carId, joint_number)
    # print(info)
    print(info[0], ": ", info[1])
    
# From this, we know that the joints connecting to our front and back wheels are 1, 3, 4, and 5,
# and our steering joints are 0 and 2.
wheel_indices = [1, 3, 4, 5]
hinge_indices = [0, 2]

0 :  b'base_to_left_hinge'
1 :  b'left_hinge_to_left_front_wheel'
2 :  b'base_to_right_hinge'
3 :  b'right_hinge_to_right_front_wheel'
4 :  b'base_to_left_back_wheel'
5 :  b'base_to_right_back_wheel'


In [6]:
while True:
    user_angle = p.readUserDebugParameter(angle)
    user_throttle = p.readUserDebugParameter(throttle)
    for joint_index in wheel_indices:
        p.setJointMotorControl2(carId, joint_index,
                                p.VELOCITY_CONTROL,
                                targetVelocity=user_throttle)
    for joint_index in hinge_indices:
        p.setJointMotorControl2(carId, joint_index,
                                p.POSITION_CONTROL, 
                                targetPosition=user_angle)
    p.stepSimulation()

KeyboardInterrupt: 